In [2]:
import os, cv2, re, random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import layers, models, optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split

In [33]:
img_width = 64
img_height = 64
RIGHT = 'imgs/'
WRONG = 'wrong/'
X = [RIGHT+i for i in os.listdir(RIGHT)] # use this for full dataset
Y = [WRONG+i for i in os.listdir(WRONG)]

In [57]:
TEST = 'test/'
test_imgs = [TEST+i for i in os.listdir(TEST)]

In [34]:
X+=Y
X

['imgs/download (1).jpg',
 'imgs/download (2).jpg',
 'imgs/download (3).jpg',
 'imgs/download (4).jpg',
 'imgs/download (5).jpg',
 'imgs/download (6).jpg',
 'imgs/download (7).jpg',
 'imgs/download (8).jpg',
 'imgs/download.jpg',
 'imgs/images (1).jpg',
 'imgs/images (2).jpg',
 'imgs/images.jpg',
 'wrong/dog.2.jpg',
 'wrong/dog.3.jpg',
 'wrong/dog.4.jpg',
 'wrong/dog.48.jpg',
 'wrong/dog.5.jpg',
 'wrong/dog.6.jpg',
 'wrong/image_0401.jpg',
 'wrong/image_0454.jpg',
 'wrong/image_0797.jpg',
 'wrong/image_1005.jpg',
 'wrong/image_1180.jpg',
 'wrong/image_1197.jpg']

In [35]:
def prepare_data(list_of_images):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """
    x = [] # images as arrays
    y = [] # labels
    
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (img_width,img_height), interpolation=cv2.INTER_CUBIC))
    
    for i in list_of_images:
        if 'imgs' in i:
            y.append(1)
        else:
            y.append(0)
        #else:
            #print('neither cat nor dog name present in images')
            
    return x, y

In [36]:
X, Y = prepare_data(X)

In [37]:
Y

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [38]:
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size=0.2, random_state=1)

In [42]:
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
batch_size = 4

In [40]:
nb_train_samples

19

In [41]:
nb_validation_samples

5

In [43]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(layers.Conv2D(32, (3, 3)))
# model.add(layers.Activation('relu'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 29, 29, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
__________

In [44]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [45]:
train_generator = train_datagen.flow(np.array(X_train), Y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(np.array(X_val), Y_val, batch_size=batch_size)

In [46]:
history = model.fit_generator(
    train_generator, 
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

Epoch 1/30
4/4 [==============================] - 24s 6s/step - loss: 1.2839 - acc: 0.3967 - val_loss: 0.7054 - val_acc: 0.2500
Epoch 2/30
4/4 [==============================] - 0s 58ms/step - loss: 0.6277 - acc: 0.6033 - val_loss: 0.7797 - val_acc: 0.2500
Epoch 3/30
4/4 [==============================] - 0s 70ms/step - loss: 0.7948 - acc: 0.3500 - val_loss: 0.5705 - val_acc: 0.7500
Epoch 4/30
4/4 [==============================] - 0s 69ms/step - loss: 0.5584 - acc: 0.9375 - val_loss: 0.4375 - val_acc: 1.0000
Epoch 5/30
4/4 [==============================] - 0s 50ms/step - loss: 0.8220 - acc: 0.5067 - val_loss: 0.5223 - val_acc: 0.7500
Epoch 6/30
4/4 [==============================] - 0s 49ms/step - loss: 0.3386 - acc: 0.9367 - val_loss: 0.5813 - val_acc: 0.7500
Epoch 7/30
4/4 [==============================] - 0s 64ms/step - loss: 0.2983 - acc: 0.8750 - val_loss: 1.3814 - val_acc: 0.2500
Epoch 8/30
4/4 [==============================] - 0s 70ms/step - loss: 0.3560 - acc: 0.7933 - val_

In [47]:
model.save_weights('model1.h5')
model.save('model_keras1.h5')

In [58]:
X_test, Y_test = prepare_data(test_imgs)

In [59]:
Y_test = []
Y_test

[]

In [60]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [61]:
test_generator = val_datagen.flow(np.array(X_test), batch_size=batch_size)
prediction_probabilities = model.predict_generator(test_generator, verbose=1)

2/2 [==============================] - 0s 39ms/step


In [62]:
prediction_probabilities

array([[0.9999529 ],
       [0.9987544 ],
       [0.9994716 ],
       [0.9999856 ],
       [0.05285164]], dtype=float32)